In [20]:
import json
import os
import pandas as pd
import ast

In [21]:
# Get list of all files in the directory
files = os.listdir('./Filtered_Output/')
jsonl_files = [file for file in files if file.endswith('.jsonl')]
print(jsonl_files)


['dataset_gpt-4_0.8.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.4.jsonl', 'dataset_Salesforce_codegen-2B-mono_1.0.jsonl', 'dataset_bigcode_starcoder_0.8.jsonl', 'dataset_gpt-3.5-turbo_0.8.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.6.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.2.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.0.jsonl', 'dataset_Salesforce_codegen25-7b-mono_1.0.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.4.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.6.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.2.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.8.jsonl', 'dataset_gpt-4_0.0.jsonl', 'dataset_gpt-3.5-turbo_0.2.jsonl', 'dataset_bigcode_starcoder_0.2.jsonl', 'dataset_gpt-3.5-turbo_0.0.jsonl', 'dataset_gpt-4_0.2.jsonl', 'dataset_gpt-4_0.6.jsonl', 'dataset_gpt-3.5-turbo_1.0.jsonl', 'dataset_bigcode_starcoder_0.6.jsonl', 'dataset_gpt-3.5-turbo_0.4.jsonl', 'dataset_gpt-4_1.0.jsonl', 'dataset_bigcode_starcoder_0.4.jsonl', 'dataset_gpt-3.5-turbo_0.6.jsonl', 'dataset

In [22]:
def is_compilable(code):
    try:
        ast.parse(code)
    except SyntaxError:
        return False
    return True

In [23]:
def search(combined_data, current_cwe, file_name):
    direct_vulnerable = 0
    indirect_vulnerable = 0
    for line in combined_data:
        if file_name in line:
            target_cwe = line.split(',')[0]
            if 'cryptography' not in target_cwe:
                target_cwe = int(target_cwe)
            if current_cwe == target_cwe:
                direct_vulnerable += 1
            else:
                indirect_vulnerable += 1
    return direct_vulnerable, indirect_vulnerable


In [24]:
for file_name in jsonl_files:
    # if 'gpt-4' not in file_name:
    #     continue
    with open('./Filtered_Output/' + file_name, 'r') as f:
        data = [json.loads(line) for line in f.readlines()]

    print(len(data))
    full_name = '_'.join(file_name.split('.jsonl')[0].split('_')[1:])
    temp = full_name.split('_')[-1]
    model_name = '_'.join(full_name.split('_')[:-1])
    print(model_name, temp)
    files = os.listdir(f'./CodeQL_Result/{full_name}')
    result_files = [file for file in files if file.endswith('.csv')]

    combined_data = []
    for result_file in result_files:
        target_cwe = result_file.split('.')[0].split('_')[-1]
        if '-' in target_cwe:
            target_cwe = target_cwe.split('-')[1]
        if '022bis' in target_cwe:
            target_cwe = '022'
        if 'cryptography'  not in target_cwe:
            target_cwe = int(target_cwe)

        with open(f'./CodeQL_Result/{full_name}/{result_file}', 'r') as f:
            lines = f.readlines()
        for line in lines:
            line = str(target_cwe)+','+model_name+','+temp+','+result_file+','+line.strip()
            combined_data.append(line)
    with open(f'./CWE_Results/{full_name}.csv', 'w') as f:
        f.write('\n'.join(combined_data)) 
    result = []
    for i in range(len(data)):
        id = data[i]['id']
        technique =  data[i]['technique']
        source = data[i]['source']
        file_name = '_'.join(id.split('_')[2:])
        current_cwe = int(id.split('_')[-2:-1][0].split('cwe')[1])

        # print(file_name, current_cwe)
        if 'gpt' in model_name:
            for j in range(len(data[i]['output']['choices'])):
                current_file_name = file_name.replace('.py', f'_{j}.py')
                direct_vulnerable, indirect_vulnerable = search(combined_data, current_cwe, current_file_name)
                # if direct_vulnerable>0:
                #     print(direct_vulnerable, indirect_vulnerable)
                is_compilable_flag = is_compilable(data[i]['output']['choices'][j]['cleared_code'])
                result.append([id,j, model_name, temp, technique, source, current_file_name,is_compilable_flag, direct_vulnerable, indirect_vulnerable])

        else:
            for j in range(len(data[i]['output'])):
                current_file_name = file_name.replace('.py', f'_{j}.py')
                direct_vulnerable, indirect_vulnerable = search(combined_data, current_cwe, current_file_name)
                # if direct_vulnerable>0:
                #     print(direct_vulnerable, indirect_vulnerable)
                is_compilable_flag = is_compilable(data[i]['output'][j]['cleared_code'])
                result.append([id,j, model_name, temp, technique, source, current_file_name,is_compilable_flag, direct_vulnerable, indirect_vulnerable])


    df = pd.DataFrame(result, columns=['id', 'index','model_name', 'Temp', 'technique', 'source', 'file_name','is_compilable', 'direct_vulnerable', 'indirect_vulnerable'])
    df.to_csv(f'./Results/{full_name}.csv', index=False)

100
gpt-4 0.8
100
Salesforce_codegen-2B-mono 0.4
100
Salesforce_codegen-2B-mono 1.0
100
bigcode_starcoder 0.8
100
gpt-3.5-turbo 0.8
100
Salesforce_codegen-2B-mono 0.6
100
Salesforce_codegen-2B-mono 0.2
100
Salesforce_codegen-2B-mono 0.0
100
Salesforce_codegen25-7b-mono 1.0
100
Salesforce_codegen25-7b-mono 0.4
100
Salesforce_codegen25-7b-mono 0.6
100
Salesforce_codegen25-7b-mono 0.2
100
Salesforce_codegen25-7b-mono 0.8
100
gpt-4 0.0
100
gpt-3.5-turbo 0.2
100
bigcode_starcoder 0.2
100
gpt-3.5-turbo 0.0
100
gpt-4 0.2
100
gpt-4 0.6
100
gpt-3.5-turbo 1.0
100
bigcode_starcoder 0.6
100
gpt-3.5-turbo 0.4
100
gpt-4 1.0
100
bigcode_starcoder 0.4
100
gpt-3.5-turbo 0.6
100
gpt-4 0.4
100
Salesforce_codegen-2B-mono 0.8
100
bigcode_starcoder 1.0


In [25]:
# Get list of all files in the directory
files = os.listdir('./Results/')
csv_files = [file for file in files if file.endswith('.csv')]
print(csv_files)

['bigcode_starcoder_0.8.csv', 'gpt-4_1.0.csv', 'Salesforce_codegen25-7b-mono_0.8.csv', 'gpt-3.5-turbo_1.0.csv', 'Salesforce_codegen-2B-mono_0.8.csv', 'gpt-4_0.2.csv', 'gpt-3.5-turbo_0.0.csv', 'gpt-3.5-turbo_0.2.csv', 'gpt-4_0.0.csv', 'gpt-4_0.4.csv', 'gpt-3.5-turbo_0.6.csv', 'gpt-3.5-turbo_0.4.csv', 'gpt-4_0.6.csv', 'gpt-3.5-turbo_0.8.csv', 'Salesforce_codegen25-7b-mono_1.0.csv', 'gpt-4_0.8.csv', 'bigcode_starcoder_1.0.csv', 'Salesforce_codegen-2B-mono_1.0.csv', 'Salesforce_codegen25-7b-mono_0.2.csv', 'Salesforce_codegen-2B-mono_0.4.csv', 'bigcode_starcoder_0.2.csv', 'Salesforce_codegen-2B-mono_0.6.csv', 'Salesforce_codegen25-7b-mono_0.4.csv', 'Salesforce_codegen-2B-mono_0.2.csv', 'bigcode_starcoder_0.6.csv', 'bigcode_starcoder_0.4.csv', 'Salesforce_codegen-2B-mono_0.0.csv', 'Salesforce_codegen25-7b-mono_0.6.csv']


In [26]:
# combine all csv files
result = []
for file_name in csv_files:
    df = pd.read_csv('./Results/' + file_name)
    result.append(df)
df = pd.concat(result)
df.to_csv('./Combined_Results.csv', index=False)